In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import time

In [38]:
prices = pd.read_csv('price_mid_wkly.csv', index_col = 0, low_memory = False)
value = pd.read_csv('new_value.csv', index_col = 0, low_memory = False)
mktcap = pd.read_csv('mkt_cap_mid_wkly.csv', index_col = 0, low_memory = False)
liquidity = pd.read_csv('turnover.csv', index_col = 0, low_memory = False)
vix = pd.read_csv('vix_contango_evalue.csv', index_col = 0, low_memory = False)
returns = pd.read_csv('mid_returns.csv', index_col = 0, low_memory = False)
# prices = prices.head()
# prices = prices.head()
# print(prices.head())
# print(factor.head())

In [21]:
dates = value.index.values.tolist()
stocks = list(prices)

#Compute tradable stocks each day 

tradable_s = {}

# for i in range(len(dates) - 1):
#     date = dates[i]
#     next_date = dates[i + 1]
#     daily_tradable_s = []
#     for stock in stocks:
#         if (not np.isnan(prices.loc[date][stock])) and (not np.isnan(prices.loc[next_date][stock])) and (not np.isnan(factor.loc[date][stock])):
#             daily_tradable_s.append(stock)
#     tradable_s[date] = daily_tradable_s
    
for i in range(len(dates) - 1):
    date = dates[i]
    next_date = dates[i + 1]
    daily_tradable_s = []
    for stock in stocks:
        if (not pd.isnull(prices.loc[date][stock])) and (not pd.isnull(prices.loc[next_date][stock])) and (not pd.isnull(value.loc[date][stock])) and (not pd.isnull(mktcap.loc[date][stock])) and (not pd.isnull(liquidity.loc[date][stock])):
            daily_tradable_s.append(stock)
    tradable_s[date] = daily_tradable_s
    print(date)

In [6]:
#Daily Holding Class

class DailyHolding(object):

    def __init__(self, date, prices, cash = 0, holdings = {}):
        self.date = date
        self.cash = cash
        self.holdings = holdings
        self.prices = prices

    def buy(self, ticker, dollars):

        #Find price
        share_price = self.prices.loc[self.date][ticker]
        shares_purchased = dollars / share_price

        #Update holdings
        if ticker in self.holdings:
            pre_buy_shares = self.holdings[ticker]
            self.holdings[ticker] = pre_buy_shares + shares_purchased
            if self.holdings[ticker] == 0:
                del self.holdings[ticker]
        else:
            self.holdings[ticker] = shares_purchased

        #Deduct from cash
        self.cash -= dollars

    def sell(self, ticker, dollars):
        #Find price
        share_price = self.prices.loc[self.date][ticker]
        shares_sold = dollars / share_price

        #Update holdings
        if ticker in self.holdings:
            pre_sell_shares = self.holdings[ticker]
            self.holdings[ticker] = pre_sell_shares - shares_sold
            if self.holdings[ticker] == 0:
                del self.holdings[ticker]
        else:
            self.holdings[ticker] = -shares_sold

        #Add to cash
        self.cash += dollars

    def clean_buy(self, ticker, dollars):

        #Find price
        share_price = self.prices.loc[self.date][ticker]
        shares_purchased = dollars / share_price

        #Update holdings
        if ticker in self.holdings:
            pre_buy_shares = self.holdings[ticker]
            self.holdings[ticker] = pre_buy_shares + shares_purchased
        else:
            self.holdings[ticker] = shares_purchased

        #Deduct from cash
        self.cash -= dollars

    def clean_sell(self, ticker, dollars):

        #Find price
        share_price = self.prices.loc[self.date][ticker]
        shares_sold = dollars / share_price

        #Update holdings
        if ticker in self.holdings:
            pre_sell_shares = self.holdings[ticker]
            self.holdings[ticker] = pre_sell_shares - shares_sold
        else:
            self.holdings[ticker] = -shares_sold

        #Add to cash
        self.cash += dollars

    def compute_value(self):
        stocks_value = 0
        for ticker in self.holdings.keys():
            #Find price
            stocks_value += self.prices.loc[self.date][ticker] * self.holdings[ticker]
        return(stocks_value + self.cash)

    def get_cash(self):
        return self.cash

    def get_all_holdings(self):
        return self.holdings

    def get_holding(self, ticker):
        return self.holdings[ticker]

    def close_everything(self):
        for ticker in self.holdings.keys():
            if self.holdings[ticker] > 0:
                self.clean_sell(ticker, (self.holdings[ticker] * self.prices.loc[self.date][ticker]))
            elif self.holdings[ticker] < 0:
                self.clean_buy(ticker, (-self.holdings[ticker] * self.prices.loc[self.date][ticker]))
        self.holdings = {}

In [7]:
print(tradable_s['1/5/2001'])

['WBS UQ Equity', 'AF UQ Equity', 'ORI UN Equity', '362614Q UN Equity', 'LGE UN Equity', 'FSCO UQ Equity', 'KSTN UQ Equity', 'DST UN Equity', 'CLF UN Equity', 'WL UN Equity', 'MTX UN Equity', 'BEC UN Equity', 'OCAS UQ Equity', 'CSL UN Equity', 'CNL UN Equity', '2857666Q UN Equity', 'DBD UN Equity', '3471Q UN Equity', 'DQE UN Equity', '4107485Q UN Equity', 'NCOG UQ Equity', 'GATX UN Equity', 'AVNT UN Equity', 'HE UN Equity', 'HMN UN Equity', 'BOCB UQ Equity', 'HUBB UN Equity', 'PPMIQ UN Equity', 'IDA UN Equity', 'WMANQ UN Equity', 'MUR UN Equity', 'OGE UN Equity', 'OLN UN Equity', 'OSGIQ UN Equity', 'NBL UN Equity', 'AFG UN Equity', 'YRK UN Equity', 'ROL UN Equity', 'CEMJQ UN Equity', 'AXLL UN Equity', 'SUP UN Equity', 'KSE UN Equity', 'FBNIQ UN Equity', 'SCG UN Equity', 'AME UN Equity', 'CNW UN Equity', 'HNI UN Equity', 'SON UN Equity', 'SRCTQ UN Equity', 'VVI UN Equity', 'CVGYQ US Equity', '1500785D UN Equity', '1683351D UQ Equity', 'MATX UQ Equity', 'GHC UN Equity', '764144Q UN Equit

In [39]:
class Backtest(object):
    
    def __init__(self, start_capital, dates, prices, value, mktcap, liquidity, vix, returns):
        self.start_capital = start_capital
        self.dates = dates
        self.positions = {}
        self.prices = prices
        self.value = value
        self.mktcap = mktcap
        self.liquidity = liquidity
        self.vix = vix
        self.returns = returns
        self.count = 0
        self.values = []
        
        # Fill first date
        self.positions[self.dates[0]] = DailyHolding(self.dates[0], self.prices, self.start_capital)
#         self.values.append(self.start_capital)
        
    # Trading logic and execution goes here
    def trade(self):
        start_time = time.time()

        old_holdings = self.positions[self.dates[0]]
        date_count = 1
        day = 0

        for date in dates[1:-1]:
#         for date in dates[1:10]:
            print(date)
            day += 1
            if not(day == 1 or day == 2):
                prevdate = dates[day - 1]
                prevprevdate = dates[day - 2]
            else:
                prevdate = dates[day]
                prevprevdate = dates[day]
            
            new_holdings = DailyHolding(date, prices, old_holdings.get_cash(), old_holdings.get_all_holdings())

            new_holdings.close_everything()
            
            self.values.append(new_holdings.get_cash())
            
            day_value = self.value.loc[date]
            day_mktcap = self.mktcap.loc[date]
            day_liquidity = self.liquidity.loc[date]
            day_vix = self.vix.loc[date]
            prevday_vix = self.vix.loc[prevdate]
            prevprevday_vix = self.vix.loc[prevprevdate]
            
#             print(day_factor)
            
            for ticker in list(day_value.index.values):
                if ticker not in tradable_s[date]:
                    del day_value[ticker]
                    
            for ticker in list(day_mktcap.index.values):
                if ticker not in tradable_s[date]:
                    del day_mktcap[ticker]
            
#             print(day_factor)
#             print(day_factor.dtypes)
            day_value = pd.to_numeric(day_value)
            #print(day_value)
            day_value_rnk = day_value.rank(ascending=True)
            #print(day_value_rnk)
            day_value_rnk = day_value_rnk.sort_values()
    
            day_mktcap = pd.to_numeric(day_mktcap)
            #print(day_mktcap)
            day_mktcap_rnk = day_mktcap.rank(ascending=True)
            #print(day_mktcap_rnk)
            day_mktcap_rnk = day_mktcap_rnk.sort_values()
            
            day_liquidity = pd.to_numeric(day_liquidity)
            #print(day_liquidity)
            day_liquidity_rnk = day_liquidity.rank(ascending=True)
            #print(day_liquidity_rnk)
            day_liquidity_rnk = day_liquidity_rnk.sort_values()
    
            day_vix = pd.to_numeric(day_vix)
            backwardation = float(day_vix.loc[:])
            
            prevday_vix = pd.to_numeric(prevday_vix)
            prevbackwardation = float(prevday_vix.loc[:])
            
            prevprevday_vix = pd.to_numeric(prevprevday_vix)
            prevprevbackwardation = float(prevprevday_vix.loc[:])
            
            price_1 = self.returns.loc[date]
            price_1 = pd.to_numeric(price_1)
            price_1 = float(price_1.loc[:])
            
            price_2 = self.returns.loc[prevdate]
            price_2 = pd.to_numeric(price_2)
            price_2 = float(price_2.loc[:])
            
            price_3 = self.returns.loc[prevprevdate]
            price_3 = pd.to_numeric(price_3)
            price_3 = float(price_3.loc[:])
            
            return_1 = price_1/price_2 - 1
            return_2 = price_2/price_3 - 1
    
            agg_rank = day_value_rnk + day_mktcap_rnk + day_liquidity_rnk
            agg_rank = agg_rank.sort_values(ascending=True)
            
            n_stocks_long = int(len(agg_rank.index) * 0.1)
            n_stocks_short = int(len(agg_rank.index) * 0.1)
            
            cash_to_each_long = new_holdings.get_cash() / n_stocks_long
            cash_to_each_short = new_holdings.get_cash() / n_stocks_short
            
            if backwardation == 1.0:
                if prevbackwardation == 1.0 and prevprevbackwardation == 1.0:
                    if return_1 > 0.0 and return_2 > 0.0:
                        for ticker in agg_rank.index[0 :  n_stocks_long]:
                            new_holdings.buy(ticker, cash_to_each_short)
            else:
                for ticker in agg_rank.index[0 :  n_stocks_long]:
                            new_holdings.buy(ticker, cash_to_each_short)
            
#             for ticker in agg_rank.index[-n_stocks_short : ]:
#                 new_holdings.sell(ticker, cash_to_each_long)
                
            print(new_holdings.compute_value())
            old_holdings = new_holdings
            date_count += 1
            self.count += 1
        print("Runtime in seconds {0:.2f}".format(time.time() - start_time))
        print("")
    
    def get_stats(self):
        self.daily_returns = pd.DataFrame(self.values).pct_change()
        self.daily_returns = self.daily_returns[1:]
        
        #Arithmetic Returns
        avg_arth_ann = self.daily_returns.mean() * 52

        #Geometric Returns
        avg_geo_ann = (stats.gmean(1 + self.daily_returns)) ** 52 - 1

        #Standard Deviation
        stdev_ann = self.daily_returns.std() * math.sqrt(52)

        #Sharpe Ratio 
        sharpe_r = avg_arth_ann / stdev_ann
        
        #Presentation
        print("Trading Statistics:")
        print(str(self.dates[0]) + " to " + str(self.dates[-1]))
        print("Annualized Arithmetic Daily Return: {0:.2%}".format(float(avg_arth_ann)))
        print("Annualized Geometric Daily Return: {0:.2%}".format(float(avg_geo_ann)))
        print("Annualized Daily Return Standard Deviation: {0:.2%}".format(float(stdev_ann)))
        print("Sharpe Ratio: {0:.2f}".format(float(sharpe_r)))
            

x = Backtest(1000000, dates, prices, value, mktcap, liquidity, vix, returns)
x.trade()
# print(x.values)
x.get_stats()

1/12/2001
1000000.0000000001
1/19/2001
1008738.232876606
1/26/2001
1023633.2921150156
2/2/2001
1045045.3241437301
2/9/2001
1045045.3241437301
2/16/2001
1056610.2624686062
2/23/2001
1033837.2321738626
3/2/2001
1033837.2321738626
3/9/2001
1033837.2321738626
3/16/2001
988295.5602782475
3/23/2001
988295.5602782475
3/30/2001
988295.5602782476
4/6/2001
962106.5339638395
4/13/2001
962106.5339638395
4/20/2001
962106.5339638395
4/27/2001
992801.6987814377
5/4/2001
1004719.83879887
5/11/2001
1018997.2904448865
5/18/2001
1018997.2904448865
5/25/2001
1018997.2904448864
6/1/2001
1013507.7364942225
6/8/2001
1025087.9878681648
6/15/2001
1014334.0423778421
6/22/2001
1014334.0423778421
6/29/2001
1032691.6758821885
7/6/2001
1010759.1116555717
7/13/2001
1027687.972279589
7/20/2001
1032480.5441845813
7/27/2001
1032480.5441845811
8/3/2001
1040391.5512495034
8/10/2001
1030351.6485413039
8/17/2001
1045268.0840771543
8/24/2001
1053730.4133194927
8/31/2001
1034559.6193811564
9/7/2001
1034559.6193811564
9/14/20

3342061.3568797517
7/21/2006
3342061.3568797517
7/28/2006
3342061.356879751
8/4/2006
3333030.585317113
8/11/2006
3285208.1390037406
8/18/2006
3405365.985024515
8/25/2006
3386107.427005143
9/1/2006
3485907.5561767984
9/8/2006
3444869.815478303
9/15/2006
3526182.6771260276
9/22/2006
3471722.099350837
9/29/2006
3515645.8416442582
10/6/2006
3548947.4888641876
10/13/2006
3627944.950917248
10/20/2006
3623430.1555299
10/27/2006
3618073.110594717
11/3/2006
3595280.599717632
11/10/2006
3643977.45753409
11/17/2006
3708616.5969304126
11/24/2006
3732296.2160638873
12/1/2006
3698436.2846081434
12/8/2006
3736487.597703122
12/15/2006
3745778.0024896404
12/22/2006
3713760.4367109416
12/29/2006
3751212.61973053
1/5/2007
3716856.462381718
1/12/2007
3792696.815574313
1/19/2007
3764852.4854836995
1/26/2007
3787258.820621598
2/2/2007
3848864.0778765203
2/9/2007
3874159.524399784
2/16/2007
3922792.951182938
2/23/2007
3972249.714895741
3/2/2007
3791938.8519443506
3/9/2007
3791938.8519443506
3/16/2007
3791938

10692016.762914238
2/17/2012
10802036.103180338
2/24/2012
10794679.025059441
3/2/2012
10541410.640184307
3/9/2012
10629765.85670438
3/16/2012
10935088.186385874
3/23/2012
10913830.360769106
3/30/2012
10964475.122685349
4/6/2012
10773570.400551293
4/13/2012
10538439.40889094
4/20/2012
10620863.172136774
4/27/2012
10826413.756655462
5/4/2012
10621440.98502024
5/11/2012
10611692.84459703
5/18/2012
10172067.654555587
5/25/2012
10385084.606683718
6/1/2012
10079077.326395584
6/8/2012
10519124.201253064
6/15/2012
10605226.672090096
6/22/2012
10693321.012238616
6/29/2012
10929591.226425353
7/6/2012
10993435.80812403
7/13/2012
10861693.06350961
7/20/2012
10753758.761156969
7/27/2012
10764682.111823471
8/3/2012
10544033.377038684
8/10/2012
10718612.5466589
8/17/2012
10844919.957318576
8/24/2012
10732865.34878431
8/31/2012
10817545.35544493
9/7/2012
11117333.351256853
9/14/2012
11557892.868195795
9/21/2012
11502109.467669131
9/28/2012
11363171.894548494
10/5/2012
11365577.783907799
10/12/2012
110

17036710.301202416
8/11/2017
16715383.861321019
8/18/2017
16551013.892485324
8/25/2017
16703060.523422407
9/1/2017
16898166.24302813
9/8/2017
16847984.288829405
9/15/2017
17139942.313143346
9/22/2017
17233950.617977154
9/29/2017
17508898.91597736
10/6/2017
17582391.516945332
10/13/2017
17500653.76868177
10/20/2017
17493465.154913817
10/27/2017
17397948.598842043
11/3/2017
17552190.773966048
11/10/2017
17714592.94571317
11/17/2017
17781537.87026576
11/24/2017
17984134.52227151
12/1/2017
18095660.151859812
12/8/2017
18012784.09495632
12/15/2017
18116941.947574176
12/22/2017
18224862.045471776
12/29/2017
18284742.39832441
1/5/2018
18336608.645838246
1/12/2018
18438157.053132284
1/19/2018
18418985.81573212
1/26/2018
18444225.10276088
2/2/2018
18019222.669361405
2/9/2018
18019222.669361405
2/16/2018
18019222.669361405
2/23/2018
18019222.669361405
3/2/2018
18045380.463550452
3/9/2018
18258380.529977407
3/16/2018
18232594.20072828
3/23/2018
17903267.003523268
3/30/2018
17903267.003523268
4/6/